Configurar o ambiente interativo do PySpark

In [1]:
from pyspark.sql import SparkSession

Cria a sessao spark

In [2]:
spark = SparkSession\
        .builder\
        .appName("twitter_transformation")\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/11/14 16:34:16 WARN Utils: Your hostname, MI0118G-ATIC resolves to a loopback address: 127.0.1.1; using 172.29.249.25 instead (on interface eth0)
23/11/14 16:34:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/14 16:34:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/14 16:34:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:750)



Define um dataframe para o caminho do DataLake 

In [4]:
df = spark.read.json("../../datalake/twitter_datascience")


In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{2, 100, 2023-11...|{[{2023-11-02T04:...|{1234567890abcdef}|  2023-11-02|
|[{72, 6, 2023-11-...|{[{2023-11-02T15:...|              null|  2023-11-02|
|[{0, 82, 2023-11-...|{[{2023-11-02T11:...|{1234567890abcdef}|  2023-11-02|
|[{11, 69, 2023-11...|{[{2023-11-02T07:...|{1234567890abcdef}|  2023-11-02|
|[{59, 9, 2023-11-...|{[{2023-11-02T16:...|{1234567890abcdef}|  2023-11-02|
|[{4, 80, 2023-11-...|{[{2023-11-02T02:...|{1234567890abcdef}|  2023-11-02|
|[{11, 99, 2023-11...|{[{2023-11-02T15:...|              null|  2023-11-02|
|[{8, 37, 2023-11-...|{[{2023-11-02T17:...|              null|  2023-11-02|
|[{75, 65, 2023-11...|{[{2023-11-02T18:...|              null|  2023-11-02|
|[{56, 72, 2023-11...|{[{2023-11-02T21:...|              null|  2023-11-02|
|[{32, 25, 2

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [7]:
from pyspark.sql import functions as f

In [8]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [10]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id", \
          "tweets.created_at", "tweets.id", \
          "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [23]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id", \
          "tweets.created_at", "tweets.id", \
          "tweets.public_metrics.*", "tweets.text")

In [12]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|        2|            100|2023-11-02T07:09:...| 99|        47|         65|         20|           30|Tweet fictício ge...|
|       32|              1|2023-11-02T16:31:...| 52|        66|         46|         76|           21|Tweet fictício cr...|
|       29|              0|2023-11-02T10:48:...| 94|        53|         62|         49|           61|Tweet fictício ge...|
|       53|             81|2023-11-02T00:27:...| 90|        72|         69|         32|           48|Tweet fictício cr...|
|       79|             62|2023-11-02T06:42:...| 15|        30|         46|         17|            3|Este é um tweet f...|
+---------+-----

In [17]:
df.select(f.explode("includes.users").alias("users")) \
  .select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [21]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [27]:
user_df.show(5)

+--------------------+
|                 col|
+--------------------+
|{2023-11-02T04:05...|
|{2023-11-02T02:44...|
|{2023-11-02T03:01...|
|{2023-11-02T21:57...|
|{2023-11-02T01:08...|
+--------------------+
only showing top 5 rows



In [28]:
user_df_test = df.select(f.explode("includes.users")).select("*")

In [29]:
user_df_test.show(10)

+--------------------+
|                 col|
+--------------------+
|{2023-11-02T04:05...|
|{2023-11-02T02:44...|
|{2023-11-02T03:01...|
|{2023-11-02T21:57...|
|{2023-11-02T01:08...|
|{2023-11-02T13:33...|
|{2023-11-02T20:27...|
|{2023-11-02T21:11...|
|{2023-11-02T14:44...|
|{2023-11-02T07:06...|
+--------------------+
only showing top 10 rows



Monta a estrutura final que será salva em um arquivo 
- método .coalesce(1): o parâmetro 1 é responsável por unir as informações que o Spark quebra em pedaços
- método .write.mode("overwrite"): o parâmetro utilizado serve para evitar problemas ao encontrar os dados já salvos
- método .json('output/tweet'): indica que salvaremos em formato JSON na estrutura de pasta definida no parâmetro

In [24]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')

In [25]:
user_df.coalesce(1).write.mode("overwrite").json('output/user')